In [1]:
import os

In [2]:
%pwd

'/Users/ntchindagiscard/Documents/end-end-mlflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/ntchindagiscard/Documents/end-end-mlflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories 

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
            ) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:

        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/ntchinda1998/recomProject.mlflow"
        )

        return model_evaluation_config

In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [10]:
from mlProject.utils.common import save_json
import dagshub


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig) -> None:

        self.config = config
    
    def eval_metrics(self, actual, pred):

        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)

        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column, "id", "title", "genre", "original_language", "overview", "release_date"], axis=1)
        test_y = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # dagshub.init(repo_owner='ntchinda1998', repo_name='recomProject', mlflow=True)
        with mlflow.start_run():
            predictions = model.predict(test_x)

            (rmse, mae, r2_score) = self.eval_metrics(test_y, predictions)

            scores = {"rmse": rmse, "mae": mae, "r2_score": r2_score}

            save_json(path =Path(self.config.metric_file_name), data=scores)
            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2_score", r2_score)

            if tracking_uri_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNet")
            else:
                mlflow.sklearn.log_model(model, "model")

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-08-19 06:09:59,382: INFO: common: Yaml file : config/config.yaml loaded successfully]
[2024-08-19 06:09:59,436: INFO: common: Yaml file : params.yaml loaded successfully]
[2024-08-19 06:09:59,471: INFO: common: Yaml file : schema.yaml loaded successfully]
[2024-08-19 06:09:59,479: INFO: common: Created directory at: artifacts]
[2024-08-19 06:09:59,500: INFO: common: Created directory at: artifacts/model_evaluation]


[2024-08-19 06:10:06,315: INFO: common: Json file saved at: artifacts/model_evaluation/metrics.json]


AttributeError: module 'mlflow' has no attribute 'sklearn_log_model'

In [ ]:
import dagshub
dagshub.init(repo_owner='ntchinda1998', repo_name='recomProject', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

In [ ]:
"https://dagshub.com/ntchinda1998/recomProject.mlflow"

In [ ]:
t 

In [ ]:
impor